In [2]:
import pandas as pd

In [3]:
#name paths for reference files
#final ------------------------
schools_file = "Resources/schools_complete.csv"
students_file = "Resources/students_complete.csv"
school_data = pd.read_csv(schools_file)
student_data = pd.read_csv(students_file)
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

#for testing only
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [4]:
#final-----------------------------
district_summary = pd.DataFrame({
    'Total Schools': [len(school_data_complete["school_name"].unique())],
    'Total Students': [school_data_complete["student_name"].count()],
    'Total Budget': [school_data_complete.budget.unique().sum()],
    'Average Math Score': [school_data_complete["math_score"].sum()/school_data_complete["math_score"].count()],
    'Average Reading Score': [school_data_complete["reading_score"].sum()/school_data_complete["reading_score"].count()],
    '% Passing Math': [len(school_data_complete[school_data_complete['math_score'] >= 70])/school_data_complete["student_name"].count()*100],
    '% Passing Reading': [len(school_data_complete[school_data_complete['reading_score'] >= 70])/school_data_complete["student_name"].count()*100],
    '% Overall Passing': [len(school_data_complete[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)])/school_data_complete["student_name"].count()*100]
})
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [5]:
#group by object variable for school summary
school_list = school_data_complete.groupby(['school_name', 'type'])

student_count = school_list['student_name'].count()
total_school_budget = school_list.budget.sum()/school_list.budget.count()
average_math_score = school_list.math_score.mean()
average_reading_score = school_list.reading_score.mean()
passing_math = school_data_complete[['student_name', 'school_name']][school_data_complete['math_score'] >= 70].groupby('school_name').count()
passing_reading  = school_data_complete[['student_name', 'school_name']][school_data_complete['reading_score'] >= 70].groupby('school_name').count()
passing_overall = (school_data_complete[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)])[['school_name', 'type', 'student_name']].groupby(['school_name', 'type']).count()

school_list = pd.merge(student_count, total_school_budget, on=['school_name', 'type'])
school_list = pd.merge(school_list, average_math_score, on=['school_name', 'type'])
school_list = pd.merge(school_list, average_reading_score, on=['school_name', 'type'])
school_list = pd.merge(school_list, passing_math, on=['school_name'])
school_list = pd.merge(school_list, passing_reading, on=['school_name'])
school_list = pd.merge(school_list, passing_overall, on=['school_name'])

school_list.head()

,student_name_x,budget,math_score,reading_score,student_name_y,student_name_x,student_name_y
school_name,,,,,,,
Bailey High School,4976,3124928.0,77.048432,81.033963,3318,4077,2719
Cabrera High School,1858,1081356.0,83.061895,83.975780,1749,1803,1697
Figueroa High School,2949,1884411.0,76.711767,81.158020,1946,2381,1569
Ford High School,2739,1763916.0,77.102592,80.746258,1871,2172,1487
Griffin High School,1468,917500.0,83.351499,83.816757,1371,1426,1330


In [53]:
school_summary = pd.DataFrame({
    'Total Students': (
        school_data_complete.groupby(['school_name', 'type'])['student_name'].count()
                      ).astype(int).map("{:,.0f}".format),
    'Total School Budget': (
        school_data_complete.groupby(['school_name', 'type'])['budget'].sum()
        /school_data_complete.groupby('school_name')['budget'].count()
                           ).astype(float).map("${:,.2f}".format),
    'Per Student Budget': (
        (school_data_complete.groupby(['school_name', 'type'])['budget'].sum()
        /school_data_complete.groupby('school_name')['budget'].count())
        /school_data_complete.groupby('school_name')['student_name'].count()
                          ).astype(float).map("${:,.2f}".format),
    'Average Math Score': (
        school_data_complete.groupby(['school_name', 'type'])['math_score'].mean()
                          ).astype(float).map("{:.2f}".format),
    'Average Reading Score': (
        school_data_complete.groupby(['school_name', 'type'])['reading_score'].mean()
                             ).astype(float).map("{:.2f}".format),
    '% Passing Math': (
        (school_data_complete[school_data_complete['math_score'] >= 70].groupby(['school_name', 'type'])['student_name'].count())
        /school_data_complete.groupby(['school_name', 'type'])['student_name'].count()
                      ).astype(float).map("{:.2%}".format),
    '% Passing Reading': (
        (school_data_complete[school_data_complete['reading_score'] >= 70].groupby(['school_name', 'type'])['student_name'].count())
        /school_data_complete.groupby(['school_name', 'type'])['student_name'].count()
                         ).astype(float).map("{:.2%}".format),
    '% Overall Passing': (
        (school_data_complete[(school_data_complete['math_score'] >= 70) 
        & (school_data_complete['reading_score'] >= 70)].groupby(['school_name', 'type'])['student_name'].count())
        /school_data_complete.groupby(['school_name', 'type'])['student_name'].count()
                        ).astype(float).map("{:.2%}".format),
})

school_summary.head()

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,type,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%


In [ ]:
school_summary = school_summary.style.format({
    'Total Students': "{:,f}"
    'Total School Budget': 
    'Per Student Budget': 
    'Average Math Score': 
    'Average Reading Score': 
    '% Passing Math': 
    '% Passing Reading': 
    '% Overall Passing'
})